In [1]:
# Import delle librerie richieste
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

In [2]:
# Trasformazioni per il training e il testing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Caricamento del Dataset

In [3]:
# Percorsi delle cartelle del dataset
data_dir = 'data\dataset\Armocromia\ARMOCROMIA 1. TEST_TRAIN'

# Creazione degli ImageFolder per il training e il testing
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'TRAIN'), data_transforms['train']),
    'test': datasets.ImageFolder(os.path.join(data_dir, 'TEST'), data_transforms['test']),
}

# Creazione dei DataLoader
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4),
}

# Numero di classi (armocromia)
num_classes = len(image_datasets['train'].classes)

Definizione del modello ResNET50 e modifica dell'ultimo stato

In [4]:
# Caricamento del modello ResNet-50 pre-addestrato
model = models.resnet50(pretrained=True)

# Congelamento dei parametri iniziali
for param in model.parameters():
    param.requires_grad = False

# Modifica dell'ultimo layer per adattarlo al numero di classi dell'armocromia
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
'''model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),   # Primo layer lineare aggiunto con 256 neuroni
    nn.ReLU(inplace=True),      # Funzione di attivazione ReLU
    nn.Dropout(0.5),            # Dropout con probabilità 0.5
    nn.Linear(256, num_classes) # Secondo layer lineare finale con num_classes output
)'''

# Spostamento del modello sulla GPU se disponibile
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

c:\Users\gians_ji5genm\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gians_ji5genm\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0


Definizione della loss function e dell'ottimizzatore

In [11]:
# Definizione della loss function (CrossEntropy) e dell'ottimizzatore (Adam)
lr=0.2
num_epochs=20
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr)

Addestramento del modello

In [12]:
# Specifica il percorso della cartella
save_dir = 'model_fine_tuning\\7th'
os.makedirs(save_dir, exist_ok=True)

In [13]:
# Funzione di addestramento del modello
def train_model(model, criterion, optimizer, num_epochs):

    # Specifica il percorso del file di log
    log_path = os.path.join(save_dir, 'training_log.txt')

    # Apri il file di log in modalità scrittura
    with open(log_path, 'w') as log_file:
        for epoch in range(num_epochs):
            log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
            log_file.write('-' * 10 + '\n')
            print(f'Epoch {epoch + 1}/{num_epochs}')
            print('-' * 10)

            # Fase di training e validation
            for phase in ['train', 'test']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                # Iterazione sui batch di dati
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # Azzera i gradienti solo se siamo nella fase di training
                    optimizer.zero_grad()

                    # Forward pass solo durante il training
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # Backward pass e ottimizzazione solo durante il training
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # Calcola le statistiche di loss e accuracy
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                # Calcola la loss media e l'accuracy media per epoch
                epoch_loss = running_loss / len(image_datasets[phase])
                epoch_acc = running_corrects.double() / len(image_datasets[phase])

                log_file.write(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}\n')
                print(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        log_file.write('\nTraining complete\n')

    return model

# Addestramento effettivo del modello
model = train_model(model, criterion, optimizer, num_epochs)

# Specifica il percorso del file di salvataggio del modello
save_path = os.path.join(save_dir, 'armocromia_resnet50.pth')

# Salvataggio del modello
torch.save(model.state_dict(), save_path)

print(f'Model saved to {save_path}')

Epoch 1/20
----------
train Loss: 166.5422, Acc: 0.1531
test Loss: 45.4660, Acc: 0.2325
Epoch 2/20
----------
train Loss: 29.5350, Acc: 0.2512
test Loss: 25.1188, Acc: 0.2169
Epoch 3/20
----------
train Loss: 26.2765, Acc: 0.2589
test Loss: 38.5086, Acc: 0.2065
Epoch 4/20
----------
train Loss: 32.2673, Acc: 0.2490
test Loss: 36.7718, Acc: 0.2519
Epoch 5/20
----------
train Loss: 40.6804, Acc: 0.2610
test Loss: 57.3024, Acc: 0.1494
Epoch 6/20
----------
train Loss: 44.0859, Acc: 0.2866
test Loss: 39.5393, Acc: 0.2805
Epoch 7/20
----------
train Loss: 36.8597, Acc: 0.2959
test Loss: 43.0378, Acc: 0.2312
Epoch 8/20
----------
train Loss: 33.3094, Acc: 0.3095
test Loss: 36.4289, Acc: 0.2831
Epoch 9/20
----------
train Loss: 33.3515, Acc: 0.3330
test Loss: 38.5615, Acc: 0.2481
Epoch 10/20
----------
train Loss: 31.1733, Acc: 0.3319
test Loss: 47.9414, Acc: 0.2325
Epoch 11/20
----------
train Loss: 41.1956, Acc: 0.2926
test Loss: 48.6320, Acc: 0.2805
Epoch 12/20
----------
train Loss: 39.65

Esplorimo il modello di interesse

In [7]:
import torch

# Percorso del file .pth
model_path = 'armocromia_resnet50.pth'

# Carica i dati dal file
checkpoint = torch.load(model_path)

# Esplora le chiavi nel checkpoint
print("Chiavi nel checkpoint:")
for key in checkpoint.keys():
    print(key)

# Se il checkpoint contiene i pesi del modello, esplora gli stati
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint

# Esplora le chiavi nei pesi del modello
print("\nChiavi nel state_dict:")
for key in state_dict.keys():
    print(key)

# Visualizza una parte dei pesi, ad esempio il primo layer
first_layer_weights = state_dict[list(state_dict.keys())[0]]
print("\nPesi del primo layer:")
print(first_layer_weights)

Chiavi nel checkpoint:
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.num_batches_tracked
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
laye